# Recurrent Language Model

In [1]:
path_to_utils = '../../utils'

dataset = 'amazon'
if dataset == 'amazon':
    path_to_data = './amazon_review_data'
else:
    path_to_data = './data'

In [2]:
import os
import sys

sys.path.append(path_to_utils)

import loading_text_and_tokenization
import torch
import numpy as np
import torch.nn as nn
import random
import math

In [3]:
USE_CUDA = True
device = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
batch_size  = 32

if dataset == 'amazon':
    batch_size = 64

### Loading Data and turning into batches

In [5]:
corpus = loading_text_and_tokenization.Corpus(path_to_data)

In [6]:
print ("Train dataset size is {}".format(len(corpus.train)))
print ("Val dataset size is {}".format(len(corpus.valid)))
print ("Test dataset size is {}".format(len(corpus.test)))

Train dataset size is 12877131
Val dataset size is 3691779
Test dataset size is 3107289


### Aside: torch.Tensor.narrow

In [7]:
dummy_tensor = torch.arange(0, 10);
print(dummy_tensor)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [8]:
dummy_tensor.narrow(0, 1, 5)

tensor([1, 2, 3, 4, 5])

In [9]:
dummy_tensor.narrow(0, 5, 4)

tensor([5, 6, 7, 8])

All we need to understand is that .narrow() is just a way to do indexing. When we do dummy_tensor.narrow(0, i, j) we are indexing dummy_tensor[i:i+j]

In [10]:
def batchify(data, bsz, random_start_idx=False):
    # calculate total number of batches that fit cleanly
    nbatch = data.size(0) // bsz
    if random_start_idx:
        start_idx = random.randint(0, data.size(0) % bsz - 1)
    else:
        start_idx = 0
        
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    # Nice thing about this: 
    # u don't need to pad since every sequence now has same length
    data = data.narrow(0, start_idx, nbatch * bsz)
    
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)


In [11]:
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, batch_size)
test_data = batchify(corpus.test, batch_size)

## RNN Model

In [12]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        
        self.encoder = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, num_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, vocab_size)

        self.init_weights()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.num_layers, bsz, self.hidden_size),
                    weight.new_zeros(self.num_layers, bsz, self.hidden_size))

In [13]:
embed_size = 200
hidden_size = 200
num_layers = 2
num_epochs = 10
lr = 20.0
dropout = 0.2
max_seq_len = 35

if dataset == 'amazon':
    num_epochs = 4;
    max_seq_len = 70


vocab_size = len(corpus.dictionary)
model = RNNModel(vocab_size, embed_size, hidden_size, num_layers, dropout).to(device)

In [14]:

def get_batch(source, i, max_seq_len):
    seq_len = min(max_seq_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


In [15]:
clip = 0.25
log_interval = 200

def repackage_hidden(h):
    """
        Wraps hidden states in new Tensors, to detach them from their history.
    """
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
def train():
    model.train()
    total_loss = 0.
    hidden = model.init_hidden(batch_size)
    
    # We shuffle train data every epoch
    train_data = train_data = batchify(corpus.train, batch_size, random_start_idx=True)
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, max_seq_len)):
        data, targets = get_batch(train_data, i, max_seq_len)
        
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch %log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // max_seq_len, lr,
                cur_loss, math.exp(cur_loss)))
            total_loss = 0


### Perplexity

Description here

In [16]:
# perplexity evaluation for a given corpus
def evaluate(data_source, max_seq_len, eval_batch_size=32):
    model.eval()
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, max_seq_len):
            data, targets = get_batch(data_source, i, max_seq_len)
            
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, vocab_size)
            
            total_loss += len(data) * criterion(output_flat, targets).item()
            hidden = repackage_hidden(hidden)
    return total_loss / len(data_source)

## Training

In [17]:
best_val_loss = np.inf
criterion = nn.CrossEntropyLoss()

In [20]:
if os.path.exists('model_'+dataset+'.pt'):
    model = torch.load('model_'+dataset+'.pt');
else:
    for epoch in range(1, num_epochs+1):
        train()
        val_loss = evaluate(val_data, max_seq_len, batch_size)
        print('-' * 89)
        print('| end of epoch {:3d} | valid loss {:5.2f} | '
                    'valid ppl {:8.2f}'.format(epoch, 
                                               val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open('model_'+dataset+'.pt', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0


| epoch   1 |   200/ 2874 batches | lr 20.00 | loss  4.79 | ppl   120.83
| epoch   1 |   400/ 2874 batches | lr 20.00 | loss  4.78 | ppl   118.96
| epoch   1 |   600/ 2874 batches | lr 20.00 | loss  4.76 | ppl   117.33
| epoch   1 |   800/ 2874 batches | lr 20.00 | loss  4.77 | ppl   117.38
| epoch   1 |  1000/ 2874 batches | lr 20.00 | loss  4.73 | ppl   113.03
| epoch   1 |  1200/ 2874 batches | lr 20.00 | loss  4.69 | ppl   109.05
| epoch   1 |  1400/ 2874 batches | lr 20.00 | loss  4.66 | ppl   105.61
| epoch   1 |  1600/ 2874 batches | lr 20.00 | loss  4.64 | ppl   103.90
| epoch   1 |  1800/ 2874 batches | lr 20.00 | loss  4.65 | ppl   104.17
| epoch   1 |  2000/ 2874 batches | lr 20.00 | loss  4.63 | ppl   103.02
| epoch   1 |  2200/ 2874 batches | lr 20.00 | loss  4.61 | ppl   100.11
| epoch   1 |  2400/ 2874 batches | lr 20.00 | loss  4.59 | ppl    98.89
| epoch   1 |  2600/ 2874 batches | lr 20.00 | loss  4.58 | ppl    97.70
| epoch   1 |  2800/ 2874 batches | lr 20.00 | loss

/scratch/sm7582/condaenvs/denoising/lib/python3.7/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   2 |   200/ 2874 batches | lr 20.00 | loss  4.54 | ppl    93.55
| epoch   2 |   400/ 2874 batches | lr 20.00 | loss  4.53 | ppl    92.33
| epoch   2 |   600/ 2874 batches | lr 20.00 | loss  4.53 | ppl    92.51
| epoch   2 |   800/ 2874 batches | lr 20.00 | loss  4.54 | ppl    93.84
| epoch   2 |  1000/ 2874 batches | lr 20.00 | loss  4.51 | ppl    91.23
| epoch   2 |  1200/ 2874 batches | lr 20.00 | loss  4.49 | ppl    88.89
| epoch   2 |  1400/ 2874 batches | lr 20.00 | loss  4.47 | ppl    87.09
| epoch   2 |  1600/ 2874 batches | lr 20.00 | loss  4.46 | ppl    86.46
| epoch   2 |  1800/ 2874 batches | lr 20.00 | loss  4.47 | ppl    86.93
| epoch   2 |  2000/ 2874 batches | lr 20.00 | loss  4.46 | ppl    86.47
| epoch   2 |  2200/ 2874 batches | lr 20.00 | loss  4.44 | ppl    84.62
| epoch   2 |  2400/ 2874 batches | lr 20.00 | loss  4.43 | ppl    83.97
| epoch   2 |  2600/ 2874 batches | lr 20.00 | loss  4.42 | ppl    83.51
| epoch   2 |  2800/ 2874 batches | lr 20.00 | loss

### Scoring The Sentence

In [21]:
def score_sentences(sent_list):
    ppl_list = []
    for sent in sent_list:
        tokens = corpus.tokenize_sentence(sent)
        
        test_sent_idx = batchify(tokens, 1)
        loss = evaluate(test_sent_idx, len(tokens), 1)
        ppl_list.append((sent, math.exp(loss)))
    return ppl_list

In [22]:
test_sentences = ['dividend yields have been bolstered by stock declines', \
             'stock bolstered declines dividend by yields have been', \
             'artificial neural networks are computing systems vaguely inspired by the biological neural networks', \
             'Cho is so cool', \
             'roberta is so cool']

score_sentences(test_sentences)

[('dividend yields have been bolstered by stock declines', 45385.34214112503),
 ('stock bolstered declines dividend by yields have been', 45385.34214112503),
 ('artificial neural networks are computing systems vaguely inspired by the biological neural networks',
  81198.96110036242),
 ('Cho is so cool', 10107.242629200658),
 ('roberta is so cool', 10107.242629200658)]

In [23]:
'Cho' in corpus.dictionary.word2idx

False

## Generation

In [24]:
n_words = 100; #number of words to generate


In [41]:
ntokens = len(corpus.dictionary)
hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
# input = torch.tensor([[corpus.dictionary.word2idx['The']]], dtype=torch.long).to(device)

In [42]:
input.shape

torch.Size([1, 1])

In [43]:
sentence = corpus.dictionary.idx2word[input.item()][1:-2] + ' '
model.eval()
with torch.no_grad():  # no tracking history
    for i in range(n_words):
        output, hidden = model(input, hidden)
        word_weights = output.squeeze().div(1.0).exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]

#         word_idx = output.topk(1)[1].item()
        
        input.fill_(word_idx)
#         input = torch.cat([ input, torch.from_numpy(np.array([[word_idx]])).type(torch.LongTensor).to(device) ], dim = 0)
        word = corpus.dictionary.idx2word[word_idx]
        
        sentence += word[1:-2] +' '

#         outf.write(word + ('\n' if i % 20 == 19 else ' '))

#         if i % 100 == 0:
#             print('| Generated {}/{} words'.format(i, n_words))

In [44]:
sentence

"loophole out at least with . good luck . a terrific little primer would need a weakness of the not nasty nonsense characteristics of a tree operation . eo 'i bought this back on the amazon 'daisey and figured out they would leverage me eo fitday isn ' t listed , but yes what ' s defective to the hks works real well . my minor complaint is the handle . the finish is pretty very quiet , though 3 / 3 dollars more , entered any accuracy ! oh , but ask a lot more at washing is why as "